In [1]:
import math
import random
import numpy as np
import pandas as pd

from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
import time

import collections
import copy

#env = gym.make('CartPole-v0').unwrapped

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# if gpu is to be used
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
device

#import os
#pathName = '/home/ubuntu/data/PEC/model_dict/ml100k/'
#if not os.path.exists(pathName):
#     os.makedirs(pathName)
#MODELPATH = pathName + 'dnn_v1.0_'

data_path = '/home/zhangxz/workspace/dataset/ml-100k/'
UIT = pd.read_csv(data_path + 'ml_header.trace')
simUsers = np.loadtxt(data_path+"simUser.csv",  delimiter=',',dtype=int)

trainUIT = UIT[UIT['day']<max(UIT['day']+1)*0.6]
validUIT = UIT[UIT['day']<max(UIT['day']+1)*0.8]

contentNum = len(UIT.i.drop_duplicates())
userNum = len(UIT.u.drop_duplicates())

rewardPara = {"alpha":0.01,"betao":1,"betal":1}
latency = [0.2,1,0.8]
Bu = 20

BATCH_SIZE = 128
GAMMA = 0.99

num_episodes = 10
EPS_START = 0.99
EPS_END = 0.01
EPS_DECAY = trainUIT.shape[0]*3
agentStep = 0

torch.cuda.empty_cache()
UIT.shape,trainUIT.shape,validUIT.shape

((132690, 4), (76252, 4), (106994, 4))

In [7]:
class ENV(object):
    def __init__(self,userNum,contentNum,latency,Bu):
        self.userNum = userNum
        self.contentNum =contentNum

        self.r = np.zeros(shape=(userNum,contentNum),dtype=int)
        self.p = np.full(shape=contentNum,fill_value = 1/userNum,dtype=np.float32)
        self.e = np.zeros(shape=contentNum,dtype=int)
        self.S = np.ones(shape=contentNum,dtype=int)
        self.l = np.array(latency,dtype=np.float32)
        

        self.B = np.full(shape=userNum,fill_value=Bu,dtype=int)

        self.pipe = collections.OrderedDict()


    #有序字典实现LRU
    def updateEgdeCache(self,action,t):
        for i in np.argwhere(action==1).squeeze(-1):
            if i in self.pipe.keys():
                self.pipe.pop(i)
            elif len(self.pipe) >= 500:
                self.e[self.pipe.popitem(last=False)[0]] = 0
            self.pipe[i] = t
            self.e[i] = 1

    
    def updateEnv(self,u,action,t):
        
        p_tmp = ((self.r[u] | action)-self.r[u])*(1/self.userNum) + self.p
        self.p = np.where(p_tmp<1-1/self.userNum,p_tmp,1-1/self.userNum)
        self.r[u] =  self.r[u] | action

        self.updateEgdeCache(action,t)

    def getStatus(self):
        return (self.r,
                self.p, 
                self.e,
                self.S,
                self.l)

    #def reset(self):
    #    self.r = np.zeros(shape=(self.userNum,self.contentNum),dtype=int)
    #    self.p = np.full(shape=self.contentNum,fill_value = 1/self.userNum)
    #    self.e = np.zeros(shape=self.contentNum)
    #    self.S = np.ones(shape=self.contentNum,dtype=int)
    #    self.l_edge = 0.1
    #    self.l_cp = 1
    #    self.B = np.full(shape=self.userNum,fill_value=15,dtype=int)
    #    self.pipe = collections.OrderedDict()

class UE_random(object):
    def __init__(self,u,env,rewardPara):
        self.u = u

        self.W = []
        self.v = np.zeros(contentNum,dtype=int)

        self.Bu = Bu
        self.contentNum = contentNum
        self.userNum = userNum

        r , p , e, S, l = env.getStatus()

        #self.lastAction = torch.zeros(size=(contentNum,),dtype=int)
        self.lastAction = np.zeros(contentNum,dtype=int)

        self.ALPHAh = rewardPara['alpha']
        self.BETAo =  rewardPara['betao']
        self.BETAl =  rewardPara['betal']             
        self.reward = 0

        self.simU = simUsers[u]
        self.lastStatusFeature = torch.from_numpy(self.statusEmbedding(r,p,e))
        #self.lastp = p.clone()
        #self.lastp = p.copy()
        
        self.lastp = p
       # self.lastr = r.clone()
        #self.lastr = r.copy()
        self.lastru = r[self.u]
        
    def updateViewContent(self,i):
        self.W.append(i)
        self.v[i] = 1

    def statusEmbedding(self,r,p,e):
        tmp = np.zeros(contentNum,dtype=np.float32)
        for simUser in self.simU:
            tmp += r[simUser]
        simUserRu = (tmp / len(self.simU))
        ru = r[self.u]
        statusFeature = np.row_stack((self.v,ru,simUserRu,p,e))
        
        return statusFeature.T.astype(np.float32)
    
    def getReward(self,lastru,lastp,ru,p,i,lastAction,S,l,e,v):
        
        #self.Rh =   self.ALPHAh * (np.log(v * p + (1-v) * (1-p)).sum() / np.log(ru * p + (1-ru) * (1-p)).sum() )

        #self.Rh = self.ALPHAh * ( 1 / ( 1 + np.exp( 0.5 * np.log( ( lastru * lastp + ( 1 - lastru ) * ( 1 - lastp ) ) / ( ru * p + ( 1 - ru ) * ( 1 - p ) ) ).sum() ) ) )

        self.Rh = - self.ALPHAh * ( np.log( ( lastru * lastp + ( 1 - lastru ) * ( 1 - lastp ) ) / ( ru * p + ( 1 - ru ) * ( 1 - p ) ) ).sum() )


        self.Ro =   self.BETAo * lastAction[i] * S[i] * ( e[i] * l[0] + ( 1 - e[i] ) * l[1] )
        
        # self.Ro =   self.BETAo * lastAction[i] * S[i] * ( 1 + e[i] * l[0] + ( 1 - e[i] ) * l[1] )

        self.Rl =   self.BETAl * ( 1 - lastAction[i] ) * S[i] * e[i] * l[2]

        #self.Rh[i] = self.Rh[i] + self.Ro + self.Rl

        #return  self.Rh.sum()
        return  (self.Rh + self.Ro + self.Rl).astype(np.float32)

    def selectAction(self,env,uit):
        
        self.updateViewContent(uit[1])
        
        r , p , e, S, l = env.getStatus()
        
        self.reward = self.getReward(self.lastru,self.lastp,r[self.u],p,self.W[-1],self.lastAction,S,l,e,self.v)

        self.lastp = p
        self.lastru = copy.copy(r[self.u])
        
        #self.lastp = p.clone()
        #self.lastr = r.clone()
        
        if self.W[-1] not in np.argwhere(self.lastAction): #local cache miss
            action = np.zeros(contentNum,dtype=int)
            action[self.W[-1]] = 1

            actionIndex = list(torch.randint(0,self.contentNum,(self.Bu,)))
        
            if self.W[-1] not in actionIndex:
                actionIndex.pop()
            for index in actionIndex:
                action[index] = 1           
            self.lastAction = action
            env.updateEnv(self.u,action,uit[2])
        else:
            action = self.lastAction # keep the cache and no request the new video
        return action


In [8]:
bestReward =  float("-inf")
env = ENV(userNum,contentNum,latency,Bu)
UEs = {}
sumReward = np.zeros(3)
loss = 0
UEHit = np.zeros(userNum)
edgeHit = 0

sumHitrate = np.zeros(UIT.shape[0] // 10000 +1)
UEHitrate = np.zeros(UIT.shape[0] // 10000 +1)
edgeHitrate = np.zeros(UIT.shape[0] // 10000 +1)
privacyReduction = np.zeros(UIT.shape[0] // 10000 +1)

for index,trace in UIT.iterrows():
    uit = trace.to_numpy()
    if uit[0] not in UEs:
        UEs[uit[0]] = UE_random(uit[0],env,rewardPara)
    ue = UEs[uit[0]]
    
    actionIndex = np.argwhere(ue.lastAction)
    if uit[1] in actionIndex:
        UEHit[uit[0]] += 1
    elif uit[1] in env.pipe.keys():
        edgeHit += 1
    ue.selectAction(env,uit)
    
    sumReward[0] += float(ue.Rh)
    sumReward[1] += float(ue.Ro)
    sumReward[2] += float(ue.Rl)
     
    if (index+1) % 10000 == 0 :
        psi = 0
        for u in UEs:
            psi += np.log(env.r[u] * env.p + (1-env.r[u]) * (1-env.p)).sum() / np.log(UEs[u].v * env.p + (1-UEs[u].v) * (1-env.p)).sum()
        print("--Time:",time.asctime( time.localtime(time.time())),"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
        print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
        print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
        print()
        sumHitrate[int(index // 10000)]   = round((edgeHit+UEHit.sum())/(index+1),5)
        UEHitrate [int(index // 10000)]   = round(UEHit.sum()/(index+1),5)
        edgeHitrate [int(index // 10000)] = round(edgeHit/(index+1),5)
        privacyReduction [int(index // 10000)] = round(float(psi)/len(UEs),5)
        
psi = 0
for u in UEs:
    psi += np.log(env.r[u] * env.p + (1-env.r[u]) * (1-env.p)).sum() / np.log(UEs[u].v * env.p + (1-UEs[u].v) * (1-env.p)).sum()
print()
print("----------------------------------------------------------------")
print("--Time:",time.asctime( time.localtime(time.time())),"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
print("----------------------------------------------------------------")
print()
sumHitrate [int(round(index / 10000,0))]  = round((edgeHit+UEHit.sum())/(index+1),5)
UEHitrate  [int(round(index / 10000,0))]  = round(UEHit.sum()/(index+1),5)
edgeHitrate[int(round(index / 10000,0))]  = round(edgeHit/(index+1),5)
privacyReduction [int(round(index / 10000,0))] = round(float(psi)/len(UEs),5)

--Time: Sat Oct 16 14:10:53 2021   Index: 9999   Loss: 0.0 --
Reward: [-8.8958e-01  3.2000e-04  2.6800e-02] total reward: -0.86246
UEHitrate: 0.0012  edgeHitrate 0.0335 sumHitrate 0.0347  privacy: 6.33933

--Time: Sat Oct 16 14:11:18 2021   Index: 19999   Loss: 0.0 --
Reward: [-7.7216e-01  2.7000e-04  2.4160e-02] total reward: -0.74773
UEHitrate: 0.00115  edgeHitrate 0.0302 sumHitrate 0.03135  privacy: 5.42722

--Time: Sat Oct 16 14:11:42 2021   Index: 29999   Loss: 0.0 --
Reward: [-7.1296e-01  2.7000e-04  2.3680e-02] total reward: -0.68902
UEHitrate: 0.0012  edgeHitrate 0.0296 sumHitrate 0.0308  privacy: 4.93379

--Time: Sat Oct 16 14:12:08 2021   Index: 39999   Loss: 0.0 --
Reward: [-6.7177e-01  2.4000e-04  2.3800e-02] total reward: -0.64774
UEHitrate: 0.00108  edgeHitrate 0.02975 sumHitrate 0.03082  privacy: 4.48328

--Time: Sat Oct 16 14:12:34 2021   Index: 49999   Loss: 0.0 --
Reward: [-6.3812e-01  2.6000e-04  2.4080e-02] total reward: -0.61378
UEHitrate: 0.00106  edgeHitrate 0.03

In [9]:
sumHitrate, UEHitrate, edgeHitrate

(array([0.0347 , 0.03135, 0.0308 , 0.03082, 0.03116, 0.03098, 0.03073,
        0.03029, 0.03067, 0.03059, 0.03075, 0.03056, 0.03055, 0.03055]),
 array([0.0012 , 0.00115, 0.0012 , 0.00108, 0.00106, 0.00102, 0.001  ,
        0.00101, 0.00103, 0.001  , 0.00105, 0.00103, 0.00102, 0.00101]),
 array([0.0335 , 0.0302 , 0.0296 , 0.02975, 0.0301 , 0.02997, 0.02973,
        0.02927, 0.02963, 0.02959, 0.0297 , 0.02952, 0.02954, 0.02954]))

In [10]:
privacyReduction

array([6.33933, 5.42722, 4.93379, 4.48328, 4.12726, 3.78263, 3.57141,
       3.36097, 3.14692, 2.96666, 2.78713, 2.64331, 2.52949, 2.4977 ])

In [12]:
class UE_None(object):
    def __init__(self,u,env,rewardPara):
        self.u = u

        self.W = []
        self.v = np.zeros(contentNum,dtype=int)

        self.Bu = Bu
        self.contentNum = contentNum
        self.userNum = userNum

        r , p , e, S, l = env.getStatus()

        #self.lastAction = torch.zeros(size=(contentNum,),dtype=int)
        self.lastAction = np.zeros(contentNum,dtype=int)

        self.ALPHAh = rewardPara['alpha']
        self.BETAo =  rewardPara['betao']
        self.BETAl =  rewardPara['betal']             
        self.reward = 0

        self.simU = simUsers[u]
        self.lastStatusFeature = torch.from_numpy(self.statusEmbedding(r,p,e))
        #self.lastp = p.clone()
        #self.lastp = p.copy()
        
        self.lastp = p
       # self.lastr = r.clone()
        #self.lastr = r.copy()
        self.lastru = r[self.u]
        
    def updateViewContent(self,i):
        self.W.append(i)
        self.v[i] = 1

    def statusEmbedding(self,r,p,e):
        tmp = np.zeros(contentNum,dtype=np.float32)
        for simUser in self.simU:
            tmp += r[simUser]
        simUserRu = (tmp / len(self.simU))
        ru = r[self.u]
        statusFeature = np.row_stack((self.v,ru,simUserRu,p,e))
        
        return statusFeature.T.astype(np.float32)
    
    def getReward(self,lastru,lastp,ru,p,i,lastAction,S,l,e,v):
        
        #self.Rh =   self.ALPHAh * (np.log(v * p + (1-v) * (1-p)).sum() / np.log(ru * p + (1-ru) * (1-p)).sum() )

        #self.Rh = self.ALPHAh * ( 1 / ( 1 + np.exp( 0.5 * np.log( ( lastru * lastp + ( 1 - lastru ) * ( 1 - lastp ) ) / ( ru * p + ( 1 - ru ) * ( 1 - p ) ) ).sum() ) ) )

        self.Rh = - self.ALPHAh * ( np.log( ( lastru * lastp + ( 1 - lastru ) * ( 1 - lastp ) ) / ( ru * p + ( 1 - ru ) * ( 1 - p ) ) ).sum() )


        self.Ro =   self.BETAo * lastAction[i] * S[i] * ( e[i] * l[0] + ( 1 - e[i] ) * l[1] )
        
        # self.Ro =   self.BETAo * lastAction[i] * S[i] * ( 1 + e[i] * l[0] + ( 1 - e[i] ) * l[1] )

        self.Rl =   self.BETAl * ( 1 - lastAction[i] ) * S[i] * e[i] * l[2]

        #self.Rh[i] = self.Rh[i] + self.Ro + self.Rl

        #return  self.Rh.sum()
        return  (self.Rh + self.Ro + self.Rl).astype(np.float32)

    def selectAction(self,env,uit):
        
        self.updateViewContent(uit[1])
        
        r , p , e, S, l = env.getStatus()
        
        self.reward = self.getReward(self.lastru,self.lastp,r[self.u],p,self.W[-1],self.lastAction,S,l,e,self.v)

        self.lastp = p
        self.lastru = copy.copy(r[self.u])
        
        #self.lastp = p.clone()
        #self.lastr = r.clone()
        
        if self.W[-1] not in np.argwhere(self.lastAction): #local cache miss
            action = np.zeros(contentNum,dtype=int)
            action[self.W[-1]] = 1      
            self.lastAction = action
            env.updateEnv(self.u,action,uit[2])
        else:
            action = self.lastAction # keep the cache and no request the new video
        return action


In [13]:
bestReward =  float("-inf")
env = ENV(userNum,contentNum,latency,Bu)
UEs = {}
sumReward = np.zeros(3)
loss = 0
UEHit = np.zeros(userNum)
edgeHit = 0

sumHitrate = np.zeros(UIT.shape[0] // 10000 +1)
UEHitrate = np.zeros(UIT.shape[0] // 10000 +1)
edgeHitrate = np.zeros(UIT.shape[0] // 10000 +1)
privacyReduction = np.zeros(UIT.shape[0] // 10000 +1)

for index,trace in UIT.iterrows():
    uit = trace.to_numpy()
    if uit[0] not in UEs:
        UEs[uit[0]] = UE_None(uit[0],env,rewardPara)
    ue = UEs[uit[0]]
    
    actionIndex = np.argwhere(ue.lastAction)
    if uit[1] in actionIndex:
        UEHit[uit[0]] += 1
    elif uit[1] in env.pipe.keys():
        edgeHit += 1
    ue.selectAction(env,uit)
    
    sumReward[0] += float(ue.Rh)
    sumReward[1] += float(ue.Ro)
    sumReward[2] += float(ue.Rl)
     
    if (index+1) % 10000 == 0 :
        psi = 0
        for u in UEs:
            psi += np.log(env.r[u] * env.p + (1-env.r[u]) * (1-env.p)).sum() / np.log(UEs[u].v * env.p + (1-UEs[u].v) * (1-env.p)).sum()
        print("--Time:",time.asctime( time.localtime(time.time())),"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
        print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
        print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
        print()
        sumHitrate[int(index // 10000)]   = round((edgeHit+UEHit.sum())/(index+1),5)
        UEHitrate [int(index // 10000)]   = round(UEHit.sum()/(index+1),5)
        edgeHitrate [int(index // 10000)] = round(edgeHit/(index+1),5)
        privacyReduction [int(index // 10000)] = round(float(psi)/len(UEs),5)
        
psi = 0
for u in UEs:
    psi += np.log(env.r[u] * env.p + (1-env.r[u]) * (1-env.p)).sum() / np.log(UEs[u].v * env.p + (1-UEs[u].v) * (1-env.p)).sum()
print()
print("----------------------------------------------------------------")
print("--Time:",time.asctime( time.localtime(time.time())),"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
print("----------------------------------------------------------------")
print()
sumHitrate [int(round(index / 10000,0))]  = round((edgeHit+UEHit.sum())/(index+1),5)
UEHitrate  [int(round(index / 10000,0))]  = round(UEHit.sum()/(index+1),5)
edgeHitrate[int(round(index / 10000,0))]  = round(edgeHit/(index+1),5)
privacyReduction [int(round(index / 10000,0))] = round(float(psi)/len(UEs),5)

--Time: Sat Oct 16 14:16:26 2021   Index: 9999   Loss: 0.0 --
Reward: [-0.05229  0.       0.2444 ] total reward: 0.19211
UEHitrate: 0.0  edgeHitrate 0.3055 sumHitrate 0.3055  privacy: 1.0

--Time: Sat Oct 16 14:16:42 2021   Index: 19999   Loss: 0.0 --
Reward: [-0.04966  0.       0.21968] total reward: 0.17002
UEHitrate: 0.0  edgeHitrate 0.2746 sumHitrate 0.2746  privacy: 1.0

--Time: Sat Oct 16 14:16:59 2021   Index: 29999   Loss: 0.0 --
Reward: [-0.04725  0.       0.22579] total reward: 0.17854
UEHitrate: 0.0  edgeHitrate 0.28223 sumHitrate 0.28223  privacy: 1.0

--Time: Sat Oct 16 14:17:16 2021   Index: 39999   Loss: 0.0 --
Reward: [-0.04593  0.       0.21646] total reward: 0.17053
UEHitrate: 0.0  edgeHitrate 0.27058 sumHitrate 0.27058  privacy: 1.0

--Time: Sat Oct 16 14:17:32 2021   Index: 49999   Loss: 0.0 --
Reward: [-0.0442  0.      0.2208] total reward: 0.1766
UEHitrate: 0.0  edgeHitrate 0.276 sumHitrate 0.276  privacy: 1.0

--Time: Sat Oct 16 14:17:50 2021   Index: 59999   Los

In [14]:
sumHitrate, UEHitrate, edgeHitrate

(array([0.3055 , 0.2746 , 0.28223, 0.27058, 0.276  , 0.26712, 0.26043,
        0.25418, 0.25832, 0.25417, 0.26074, 0.26049, 0.25458, 0.25491]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0.3055 , 0.2746 , 0.28223, 0.27058, 0.276  , 0.26712, 0.26043,
        0.25417, 0.25832, 0.25417, 0.26074, 0.26049, 0.25458, 0.25491]))

In [15]:
privacyReduction

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])